# Classification supervisée

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sps

# Plot et Display
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

# Scikit learn
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier 

#Lecture des données 
import pickle

In [ ]:
#path_test = "../../4A/Projet/Ensemble de test/"
path_test = "../Donnees_projet/Ensemble_de_test/"

In [ ]:
with open('liste_propre', 'rb') as fichier:
    mon_depickler = pickle.Unpickler(fichier)
    liste_propre, ind_recupere = mon_depickler.load()

In [ ]:
n = len(liste_propre)
liste_appr = list(np.asarray(liste_propre)[np.asarray(ind_recupere) <= 299])
liste_test = list(np.asarray(liste_propre)[np.asarray(ind_recupere) > 299])
n_appr = len(liste_appr)
n_test = len(liste_test)

In [ ]:
anom = [False] * n_test
type_anom = [-1] * n_test
loc = [-1] * n_test

anom[73] = True
type_anom[73] = 1
loc[73] = 200

anom[96] = True
type_anom[96] = 1
loc[96] = 300

anom[36] = True
type_anom[36] = 1
loc[36] = 400

anom[32] = True
type_anom[32] = 1
loc[32] = 300

anom[33] = True
type_anom[33] = 1
loc[33] = 200

anom[107] = True
type_anom[107] = 2
loc[107] = 600

anom[60] = True
type_anom[60] = 2
loc[60] = 400

anom[113] = True
type_anom[113] = 2
loc[113] = 400

anom[9] = True
type_anom[9] = 2
loc[9] = 200

anom[11] = True
type_anom[11] = 2
loc[11] = 300

anom[53] = True
type_anom[53] = 3
loc[53] = 400

anom[114] = True
type_anom[114] = 3
loc[114] = 400

anom[14] = True
type_anom[14] = 3
loc[14] = 500

anom[79] = True
type_anom[79] = 3
loc[79] = 400

anom[29] = True
type_anom[29] = 3
loc[29] = 300

anom[27] = True
type_anom[27] = 4
loc[27] = 300

anom[121] = True
type_anom[121] = 4
loc[121] = 300

anom[5] = True
type_anom[5] = 4
loc[5] = 400

anom[89] = True
type_anom[89] = 4
loc[89] = 200

anom[99] = True
type_anom[99] = 4
loc[99] = 100  

anom = [False] * n_appr + anom
type_anom = [-1] * n_appr + type_anom
loc = [-1] * n_appr + loc

In [ ]:
ind_anom = list(np.where(np.asarray(anom))[0]) # Indices des anomalies

In [ ]:
def df_fenetres(liste_comp, N, anom, loc):
    serie = []
    #origine = []
    ind_debut = []
    valeurs = []
    anom_fen = []
    for i, val, loc_i in zip(ind_recupere, liste_comp, loc):
        # Liste des fenêtres de l'enregistrement
        fenetres = [val[i * N:(i + 1) * N] for i in range((len(val) + N - 1) // N ) if len(val[i * N:(i + 1) * N]) == N]
        nb_fen = len(fenetres)
        valeurs += fenetres
        anom_fen_i = [False] * nb_fen
        pos_anom_deb_fen = loc_i // N
        pos_anom_fin_fen = (loc_i + 55) // N
        if loc_i > -1:
            try:
                for pos in range(pos_anom_deb_fen, pos_anom_fin_fen + 1):
                    anom_fen_i[pos] = True

            except IndexError:
                pass
                """Cette erreur est levée lorsque l'anomalie se trouve en fin de série, dans une plage qui n'a pas été capturée 
                par une des fenêtres. On peut donc l'ignorer et continuer le traitement. Ce cas est de toute façon très rare
                dans nos données."""
        anom_fen += anom_fen_i
        ind_debut += list(range(0, nb_fen * N, N))
        serie += [i] * nb_fen
        """if i < 300:
            origine += ["appr"] * nb_fen
        else:
            origine += ["test"] * nb_fen"""
        
    
    df = pd.DataFrame(data={"id" : list(range(len(serie))), "serie" : serie, "ind_debut" : ind_debut, "valeurs" : valeurs, "anom" : anom_fen})
        
    return df.set_index(["id"])

In [ ]:
N = 80
data_fenetres = df_fenetres(liste_propre, N, anom, loc)

In [ ]:
ind_anom = list(data_fenetres[data_fenetres["anom"]]["serie"].values)

In [ ]:
# Toutes les fenetres des séries contenant une anomalie
data_ajout_anom = data_fenetres.loc[data_fenetres["serie"].isin(ind_anom)] 
# On met un indice d'une grande puisance de 10 pour identifier plus facilement les fenetres ajoutees
offset = 10 ** (int(np.log10(data_fenetres.index.values[-1])) + 1)

df_supervise = data_fenetres
for i in range(1,2):
    data_ajout_anom.index += offset
    data_ajout_anom.serie += offset
    df_supervise = pd.concat([df_supervise, data_ajout_anom])
    
# df_supervise contient les données d'apprentissage + les données de tests + les séries anormales recopiées 4 fois
# les séries recopiées ont pour indice leur indice de départ + k*offset pour k=1,..,4
df_supervise.tail(50)

In [ ]:
def plot_anom_type(Type):
    for num in df_supervise.index.values:
        TS = df_supervise.loc[num]
        loc = TS["loc"]
        serie = TS["valeurs"]

        if TS["type"] == Type: 
            fig, ax = plt.subplots(figsize=(12, 7))
            plt.title("Série numéro " + str(num) + ", type " + str(Type), size=20, color='r', fontweight='bold')

            ax.plot(range(loc), serie[: loc], 'b')
            ax.plot(range(loc - 1, loc + 56), serie[loc - 1 : loc + 56], 'r')
            ax.plot(range(loc + 55, len(serie)), serie[loc + 55 :], 'b')

        plt.show()

In [ ]:
try:
    for T in range(1, 5):
        plot_anom_type(T)
except KeyError:
    print("Marche plus avec le nouveau DataFrame, à adapter si besoin.")

## Méthodes de Classification

## Random Forest

### Calcul des features

In [ ]:
df_supervise["min"] = list(map(min, df_supervise.valeurs))
df_supervise["max"] = list(map(max, df_supervise.valeurs))
df_supervise["mean"] = list(map(np.mean, df_supervise.valeurs))
df_supervise["std"] = list(map(np.std, df_supervise.valeurs))
df_supervise["skew"] = list(map(sps.skew, df_supervise.valeurs))
df_supervise["kurt"] = list(map(sps.kurtosis, df_supervise.valeurs))
df_supervise["energy"] = list(map(lambda x : sum(np.asarray(x)**2)/len(x), df_supervise.valeurs))
df_supervise["average_cross"] = list(map(lambda x : sum(1*(x > np.mean(x)))/len(x), df_supervise.valeurs))

In [ ]:
scaler = preprocessing.StandardScaler()
names_features = ["min", "max", "mean", "std", "skew", "kurt", "energy", "average_cross"]
for col in names_features:
    df_supervise[col] = scaler.fit_transform(df_supervise[col].values.reshape(-1, 1))

In [ ]:
taille_test = df_supervise.shape[0] // 3
X_train, X_test, Y_train, Y_test = train_test_split(df_supervise[names_features], df_supervise["anom"], test_size=taille_test)

In [ ]:
# définition des paramètres
forest = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=None, min_samples_split=2,
                                min_samples_leaf=1, max_features='auto', max_leaf_nodes=None, bootstrap=True, oob_score=True)
# apprentissage
rfFit = forest.fit(X_train, Y_train)
print(1 - rfFit.oob_score_)

In [ ]:
# erreur de prévision sur le test
1 - rfFit.score(X_test, Y_test)

In [ ]:
param = [{"max_features" : list(range(2, len(names_features) + 1))}]
rf = GridSearchCV(RandomForestClassifier(n_estimators=500), param, cv=5, n_jobs=-1)
rfOpt = rf.fit(X_train, Y_train)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - rfOpt.best_score_,rfOpt.best_params_))

In [ ]:
# erreur de prévision sur le test
1 - rfOpt.score(X_test,Y_test)

In [ ]:
# prévision
y_chap = rfOpt.predict(X_test)
# matrice de confusion
# Options pour normalize : all, index, column
table_norm_index = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize=False) # margins=True, margins_name="Total")
table_norm_col = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="columns")
sns.set(font_scale=1.4)#for label size
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 5))
sns.heatmap(table_norm_index, annot=True, cmap="Blues", ax=ax[0], annot_kws={"size": 16}) # font size
ax[0].set_title("Matrice de confusion normalisée \n selon les lignes")
sns.heatmap(table_norm_col, annot=True, cmap="Blues", ax=ax[1], annot_kws={"size": 16}) # font size
ax[1].set_title("Matrice de confusion normalisée \n selon les colonnes")
plt.show()

Commentaire : 1,2% d'erreurs, c'est très faible mais si on regarde la matrice de confusion on voit que la prévision n'est en fait pas si efficace. En effet on observe beaucoup de faux négatifs, envion un tiers. On a pas contre quasi aucun faux positif.

On teste la même chose mais sans dupliquer les anomalies.

In [ ]:
data_fenetres["min"] = list(map(min, data_fenetres.valeurs))
data_fenetres["max"] = list(map(max, data_fenetres.valeurs))
data_fenetres["mean"] = list(map(np.mean, data_fenetres.valeurs))
data_fenetres["std"] = list(map(np.std, data_fenetres.valeurs))
data_fenetres["skew"] = list(map(sps.skew, data_fenetres.valeurs))
data_fenetres["kurt"] = list(map(sps.kurtosis, data_fenetres.valeurs))
data_fenetres["energy"] = list(map(lambda x : sum(np.asarray(x)**2)/len(x), data_fenetres.valeurs))
data_fenetres["average_cross"] = list(map(lambda x : sum(1*(x > np.mean(x)))/len(x), data_fenetres.valeurs))

In [ ]:
scaler = preprocessing.StandardScaler()
names_features = ["min", "max", "mean", "std", "skew", "kurt", "energy", "average_cross"]
for col in names_features:
    data_fenetres[col] = scaler.fit_transform(data_fenetres[col].values.reshape(-1, 1))

In [ ]:
taille_test = data_fenetres.shape[0] // 3
X_train, X_test, Y_train, Y_test = train_test_split(data_fenetres[names_features], data_fenetres["anom"], test_size=taille_test)

In [ ]:
# définition des paramètres
forest = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1,
                                max_features='auto', max_leaf_nodes=None, bootstrap=True, oob_score=True)
# apprentissage
rfFit = forest.fit(X_train, Y_train)
print(1 - rfFit.oob_score_)

In [ ]:
# erreur de prévision sur le test
1 - rfFit.score(X_test, Y_test)

In [ ]:
param = [{"max_features" : list(range(2, len(names_features) + 1))}]
rf = GridSearchCV(RandomForestClassifier(n_estimators=500), param, cv=5, n_jobs=-1)
rfOpt = rf.fit(X_train, Y_train)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - rfOpt.best_score_, rfOpt.best_params_))

In [ ]:
# erreur de prévision sur le test
1 - rfOpt.score(X_test, Y_test)

In [ ]:
# prévision
y_chap = rfFit.predict(X_test)
# matrice de confusion
# Options pour normalize : all, index, column
table_norm_index = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="index") # margins=True, margins_name="Total")
table_norm_col = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="columns")
sns.set(font_scale=1.4)#for label size
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 5))
sns.heatmap(table_norm_index, annot=True, cmap="Blues", ax=ax[0], annot_kws={"size": 16}) # font size
ax[0].set_title("Matrice de confusion normalisée \n selon les lignes")
sns.heatmap(table_norm_col, annot=True, cmap="Blues", ax=ax[1], annot_kws={"size": 16}) # font size
ax[1].set_title("Matrice de confusion normalisée \n selon les colonnes")
plt.show()

Commentaire : Le modèle répond quasi systématiquement Non, et donc se trompe très peu. C'est parfaitement inexploitable, il faut garder la version en dupliquant les anomalies ou trouver une autre idée.

### Coefficients de Fourier

In [ ]:
from scipy.fftpack import fft
fftCoeff =[]
for x in df_supervise["valeurs"]:
    mx = np.mean(x)
    x_centre = x - mx
    #Apply fast Fourier transform
    coeffsfft = np.abs(fft(x_centre))
    coeffsfft_flatten = np.hstack(coeffsfft)
    fftCoeff.append(coeffsfft_flatten)
    
fftCoeff = np.array(fftCoeff)
print(fftCoeff.shape)

#Coefficients seuillés
prop_a_garder = 0.1
nb_coeffs = int(fftCoeff.shape[1] * prop_a_garder)
somme = np.sum(fftCoeff**2, axis=0)
fftCoeff_seuil = np.zeros((0, nb_coeffs))
ind_grands = np.argsort(somme)[-nb_coeffs :]
fftCoeff_seuil = fftCoeff[:, ind_grands]

print(fftCoeff_seuil.shape)

In [ ]:
df_Fourier = pd.DataFrame(fftCoeff)
df_Fourier_seuil = pd.DataFrame(fftCoeff_seuil)

In [ ]:
for col in df_Fourier.columns :
    df_Fourier[col] = scaler.fit_transform(df_Fourier[col].values.reshape(-1, 1))
for col in df_Fourier_seuil.columns :
    df_Fourier_seuil[col] = scaler.fit_transform(df_Fourier_seuil[col].values.reshape(-1, 1))

### Coefficients d'ondelettes

In [ ]:
import pywt
from pywt import wavedec
from statsmodels.robust import mad

wavelist=['haar', 'db2']

wavelist=['haar', 'db2'] 

Coeff_ond_haar = []
Coeff_ond_db2 = []

for x in df_supervise["valeurs"]:
    #Apply wavelet decomposition
    coeffs_haar = pywt.wavedec(x, wavelist[0], level=4) 
    coeffs_flatten_haar, coeff_slices_haar = pywt.coeffs_to_array(coeffs_haar)
    Coeff_ond_haar.append(coeffs_flatten_haar)
    #Apply wavelet decomposition
    coeffs_db2 = pywt.wavedec(x, wavelist[1], level=4) 
    coeffs_flatten_db2, coeff_slices_db2 = pywt.coeffs_to_array(coeffs_db2)
    Coeff_ond_db2.append(coeffs_flatten_db2)
    
    
Coeff_ond_haar = np.array(Coeff_ond_haar)
Coeff_ond_db2 = np.array(Coeff_ond_db2)

print(Coeff_ond_haar.shape, Coeff_ond_db2.shape)

In [ ]:
df_ond_haar = pd.DataFrame(Coeff_ond_haar)
df_ond_db2 = pd.DataFrame(Coeff_ond_db2)

In [ ]:
#for col in df_ond_haar.columns :
#    df_ond_haar[col] = scaler.fit_transform(df_ond_haar[col].values.reshape(-1, 1))
#for col in df_ond_db2.columns :
#    df_ond_db2[col] = scaler.fit_transform(df_ond_db2[col].values.reshape(-1, 1))

### Visualisation des résultats

In [ ]:
# Interactive display
from ipywidgets import interact, widgets, interactive, fixed, interact_manual
from IPython.display import display

df_liste = ["Features",
            "Ondelettes_haar",
            "Ondelettes_db2",
            "Coefficients de Fourier",
            "Coefficients de Fourier seuillés"
           ]

dict_df = {"Features" : df_supervise[names_features],
           "Ondelettes_haar" : df_ond_haar,
           "Ondelettes_db2" : df_ond_db2,
           "Coefficients de Fourier" : df_Fourier,
           "Coefficients de Fourier seuillés" : df_Fourier_seuil
          }

taille_max = df_supervise.shape[0]
print(taille_max)

In [ ]:
df_supervise

In [ ]:
@interact(Choix_df=df_liste, taille_test=widgets.IntSlider(min=100, max=taille_max - 100, step=100, 
                                                           value=taille_max//3, continuous_update=False), 
          button=widgets.ToggleButton(description="Refresh"))

def RandomForest(Choix_df, taille_test, button):
    df_coeff = dict_df[Choix_df]
    #print("Proportion de fenêtres utilisées pour l'apprentissage : ", 1 - taille_test / df_coeff.shape[0])
    X_train, X_test, Y_train, Y_test = train_test_split(df_coeff, df_supervise["anom"], test_size=taille_test)
    
    # définition des paramètres
    
    # apprentissage
    
    max_feat = 5
    param = [{"max_features" : list(range(2, min(max_feat, df_coeff.shape[1]) + 1))}]
    rf = GridSearchCV(RandomForestClassifier(n_estimators=500), param, cv=5, n_jobs=-1)

    rfOpt = rf.fit(X_train, Y_train)
    
    #print("Erreur Out of bag : ", 1 - rfOpt.oob_score_)  # N'existe pas sur les objets GridSearchCV, hélas
    # erreur de prévision sur le test
    #print("Erreur de prévision sur le test : ", 1 - rfOpt.score(X_test, Y_test))
    # paramètre optimal
    #print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - rfOpt.best_score_, rfOpt.best_params_))
    
    forest = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=None, min_samples_split=2,
                                    min_samples_leaf=1, max_features=rfOpt.best_params_['max_features'], 
                                    max_leaf_nodes=None, bootstrap=True, oob_score=True)
    rfFit = forest.fit(X_train, Y_train)
    
    # prévision
    y_chap = rfOpt.predict(X_test)
    
    
    df_supervise["pred"] = np.array([False] * df_supervise.shape[0])
    for ind, ind_fen in enumerate(Y_test.index):
        df_supervise.loc[ind_fen, "pred"] = y_chap[ind]
        
    ind_series_anom = np.unique(df_supervise[df_supervise["pred"]]["serie"].values)
    print("Séries contenant une anomalie : ", ind_series_anom)
    # matrice de confusion

    table = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize=False)
    table_norm = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="all")
    table_norm_index = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="index")
    table_norm_col = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="columns")
    sns.set(font_scale=1.4)
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))
    sns.heatmap(table, annot=True, cmap="Blues", ax=ax[0, 0], annot_kws={"size": 16})
    ax[0, 0].set_title("Matrice de confusion brute")
    sns.heatmap(table_norm, annot=True, cmap="Blues", ax=ax[0, 1], annot_kws={"size": 16})
    ax[0, 1].set_title("Matrice de confusion normalisée")
    sns.heatmap(table_norm_index, annot=True, cmap="Blues", ax=ax[1, 0], annot_kws={"size": 16})
    ax[1, 0].set_title("Matrice de confusion normalisée selon les lignes")
    sns.heatmap(table_norm_col, annot=True, cmap="Blues", ax=ax[1, 1], annot_kws={"size": 16})
    ax[1, 1].set_title("Matrice de confusion normalisée selon les colonnes")
    plt.show()

    for num in ind_series_anom:

        fig, ax = plt.subplots(figsize=(8, 5))
        TS = df_supervise[(df_supervise["serie"] == num)]["valeurs"].values
        anoms = df_supervise[df_supervise["serie"] == num]["pred"].values
        title_obj = plt.title("Série numéro " + str(num % offset), size=25)
        for (i, x), anom in zip(enumerate(TS), anoms):

            if i > 0:  # Raccordement
                ax.plot([N * i - 1, N * i], [last, x[0]], "br"[int(anom)])

            ax.plot(range(N * i, N * (i + 1)),  x, "br"[int(anom)])
            last = x[-1]
            if anom:
                plt.setp(title_obj, color='r', fontweight="bold")
    
        plt.show()

**Commentaires :**

Très peu d'erreur avec les coefficients d'ondelettes (haar ou db2) et même parfois 0 erreur.

Avec les coefficients de Fourier : résulats quasi identiques à ceux de Features, parfois meilleurs, parfois pires.

### A FINIR

In [ ]:
# RandomForest sur coefficients d'ondelettes db2

X_train, X_test, Y_train, Y_test = train_test_split(Coeff_ond_db2, df_supervise["anom"],
                                                    test_size=Coeff_ond_db2.shape[1] // 3)

max_feat = 16
param = [{"max_features" : list(range(2, min(max_feat, Coeff_ond_db2.shape[1]) + 1))}]
rf = GridSearchCV(RandomForestClassifier(n_estimators=500), param, cv=5, n_jobs=-1)
rfOpt = rf.fit(X_train, Y_train)

# erreur de prévision sur le test
print("Erreur de prévision sur le test : ", 1 - rfOpt.score(X_test, Y_test))
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - rfOpt.best_score_, rfOpt.best_params_))

# forest optimal
forest = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=None, min_samples_split=2,
                                min_samples_leaf=1, max_features=2, max_leaf_nodes=None, bootstrap=True,
                                oob_score=True)
rfFit = forest.fit(X_train, Y_train)

In [ ]:
print("Importance de chaque coefficient : ", rfFit.feature_importances_)

In [ ]:
y_chap = rfOpt.predict(X_test)
y_chap

In [ ]:
importance = rfFit.feature_importances_

In [ ]:
# coefficients triés par ordre d'importance
np.argsort(rfFit.feature_importances_)[::-1] #dans l'ordre

In [ ]:
# on ne garde que les 10 coefficients les plus importants pour reconstruire un signal avec anomalie
seuil = np.sort(importance)[-10]
valeurs = df_ond_db2.loc[6, :].values
valeurs_seuil = np.array([v if imp >= seuil else 0 for imp, v in zip(importance, valeurs)])
valeurs_seuil

In [ ]:
plt.plot(pywt.waverecn(pywt.array_to_coeffs(valeurs_seuil, coeff_slices), 'db2'))

In [ ]:
#le signal d'origine
plt.plot(np.array(df_supervise.loc[6, "valeurs"]))

Tester regression logistique
Pouvoir visualiser l'anomalie
Tester lissage spline ou supprimer les niveaux de détails les plus fins en ondelettes


## Regression Logistique 

### Calcul des features

In [ ]:
df_supervise["min"] = list(map(min, df_supervise.valeurs))
df_supervise["max"] = list(map(max, df_supervise.valeurs))
df_supervise["mean"] = list(map(np.mean, df_supervise.valeurs))
df_supervise["std"] = list(map(np.std, df_supervise.valeurs))
df_supervise["skew"] = list(map(sps.skew, df_supervise.valeurs))
df_supervise["kurt"] = list(map(sps.kurtosis, df_supervise.valeurs))
df_supervise["energy"] = list(map(lambda x : sum(np.asarray(x)**2)/len(x), df_supervise.valeurs))
df_supervise["average_cross"] = list(map(lambda x : sum(1*(x > np.mean(x)))/len(x), df_supervise.valeurs))

In [ ]:
scaler = preprocessing.StandardScaler()
names_features = ["min", "max", "mean", "std", "skew", "kurt", "energy", "average_cross"]
for col in names_features:
    data_fenetres[col] = scaler.fit_transform(data_fenetres[col].values.reshape(-1, 1))

In [ ]:
taille_test = df_supervise.shape[0] // 3
X_train, X_test, Y_train, Y_test = train_test_split(df_supervise[names_features], df_supervise["anom"], test_size=taille_test)

#### Optimisation Lasso

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# Grille de valeurs du paramètre de pénalisaiton
param=[{"C": [0.5, 1, 5, 10, 12, 15, 30]}]
logitL = GridSearchCV(LogisticRegression(penalty="l1"), param, cv=5, n_jobs=-1)
logitLasso = logitL.fit(X_train, Y_train)
# Sélection du paramètre optimal
logitLasso.best_params_["C"]
print("Meilleur score (apprentissage) = %f, Meilleur paramètre = %s" % (1.-logitLasso.best_score_,logitLasso.best_params_))

In [ ]:
# définition des estimateurs
logit= LogisticRegression(penalty="l1")
logitOpt=logit.fit(X_train, Y_train)
# erreur sur l'échantillon test
1-logitOpt.score(X_test, Y_test)

Le modèle "optimal" obtenu est utilisé pour prédire l'échantillon test et estimer ainsi, sans biais, une erreur de prévision.

La matrice de confusion croise les abomalies prédites avec celles effectivement présentes.

In [ ]:
# prévision
y_chap = logitOpt.predict(X_test)
df_supervise["pred"] = np.concatenate((np.array([False] * (df_supervise.shape[0] - len(y_chap))), y_chap))
ind_series_anom = np.unique(df_supervise[df_supervise["pred"]]["serie"].values)
# matrice de confusion
table = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize=False) # margins=True, margins_name="Total")
table_norm = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="all")
table_norm_index = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="index") # margins=True, margins_name="Total")
table_norm_col = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="columns")
sns.set(font_scale=1.4)  #for label size
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))
sns.heatmap(table, annot=True, cmap="Blues", ax=ax[0, 0], annot_kws={"size": 16})  # font size
ax[0, 0].set_title("Matrice de confusion brute")
sns.heatmap(table_norm, annot=True, cmap="Blues", ax=ax[0, 1], annot_kws={"size": 16}) # font size
ax[0, 1].set_title("Matrice de confusion normalisée")
sns.heatmap(table_norm_index, annot=True, cmap="Blues", ax=ax[1, 0], annot_kws={"size": 16}) # font size
ax[1, 0].set_title("Matrice de confusion normalisée selon les lignes")
sns.heatmap(table_norm_col, annot=True, cmap="Blues", ax=ax[1, 1], annot_kws={"size": 16}) # font size
ax[1, 1].set_title("Matrice de confusion normalisée selon les colonnes")
plt.show()

In [ ]:
# Coefficients
LassoOpt=LogisticRegression(penalty="l1",C=12)
LassoOpt=LassoOpt.fit(X_train, Y_train)
# Récupération des coefficients
vect_coef=np.matrix.transpose(LassoOpt.coef_)
vect_coef=vect_coef.ravel()
#Affichage des 25 plus importants
coef=pd.Series(abs(vect_coef),index=X_train.columns).sort_values(ascending=False)
print(coef)

In [ ]:
plt.figure(figsize=(7,4))
coef.plot(kind='bar')
plt.title('Coeffients')
plt.tight_layout()
plt.show()

Les coefficients les plus importants dans le cas de features sont ceux de average_cross et skew. 

#### Optimisation ridge

In [ ]:
# Grilles de valeurs du paramètre de pénalisation
param=[{"C":[0.5, 1, 5, 10, 12, 15, 30]}]
logitR = GridSearchCV(LogisticRegression(penalty="l2"), param, cv=5, n_jobs=-1)
logitRidge = logitR.fit(X_train, Y_train)  
# Sélection du paramètre optimal
logitRidge.best_params_["C"]
print("Meilleur score = %f, Meilleur paramètre = %s" % (1.-logitRidge.best_score_, logitRidge.best_params_))

In [ ]:
# Prévision
y_chap = logitRidge.predict(X_test)
df_supervise["pred"] = np.concatenate((np.array([False] * (df_supervise.shape[0] - len(y_chap))), y_chap))
ind_series_anom = np.unique(df_supervise[df_supervise["pred"]]["serie"].values)
# matrice de confusion
table = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize=False) # margins=True, margins_name="Total")
table_norm = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="all")
table_norm_index = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="index") # margins=True, margins_name="Total")
table_norm_col = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="columns")
sns.set(font_scale=1.4)  #for label size
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))
sns.heatmap(table, annot=True, cmap="Blues", ax=ax[0, 0], annot_kws={"size": 16})  # font size
ax[0, 0].set_title("Matrice de confusion brute")
sns.heatmap(table_norm, annot=True, cmap="Blues", ax=ax[0, 1], annot_kws={"size": 16}) # font size
ax[0, 1].set_title("Matrice de confusion normalisée")
sns.heatmap(table_norm_index, annot=True, cmap="Blues", ax=ax[1, 0], annot_kws={"size": 16}) # font size
ax[1, 0].set_title("Matrice de confusion normalisée selon les lignes")
sns.heatmap(table_norm_col, annot=True, cmap="Blues", ax=ax[1, 1], annot_kws={"size": 16}) # font size
ax[1, 1].set_title("Matrice de confusion normalisée selon les colonnes")
plt.show()

In [ ]:
#Courbe ROC, Elle est utile pour comparer Lasso et Ridge 
from sklearn.metrics import roc_curve
listMethod=[["Lasso", logitLasso], ["Ridge", logitRidge]]

for (name, method) in listMethod:
    probas = method.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(Y_test, probas[:,1])
    plt.plot(fpr, tpr, lw=1, label=str(name))
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc="best")
plt.show()

### Coefficient de Fourier et d'ondelettes 

#### Visulation par intéract 

#### Optimisation Lasso

In [ ]:
@interact(Choix_df=df_liste, taille_test=widgets.IntSlider(min=100, max=taille_max - 100, step=100, 
                                                           value=taille_max//3, continuous_update=False), 
          button=widgets.ToggleButton(description="Refresh"))

def RLLasso(Choix_df, taille_test, button):
    df_coeff = dict_df[Choix_df]
    print("Proportion de fenêtres utilisées pour l'apprentissage : ", 1 - taille_test / df_coeff.shape[0])
    X_train, X_test, Y_train, Y_test = train_test_split(df_coeff, df_supervise["anom"], test_size=taille_test)
    
    param = [{"C": [0.5,1,5,10,12,15,30]}]
    logitL = GridSearchCV(LogisticRegression(penalty="l1"), param, cv=5, n_jobs=-1)
    logitLasso = logitL.fit(X_train, Y_train)
    # Sélection du paramètre optimal
    logitLasso.best_params_["C"]
    print("Meilleur score (apprentissage) = %f, Meilleur paramètre = %s" %
          (1.-logitLasso.best_score_,logitLasso.best_params_))
    
    # définition des estimateurs
    logit= LogisticRegression(penalty="l1")
    logitOpt = logit.fit(X_train, Y_train)
    # erreur sur l'échantillon test
    print("Erreur sur l'échantillon test : ", 1-logitOpt.score(X_test, Y_test))

    
    # prévision
    y_chap = logitOpt.predict(X_test)
    df_supervise["pred"] = np.array([False] * df_supervise.shape[0])
    for ind, ind_fen in enumerate(Y_test.index):
        df_supervise.loc[ind_fen, "pred"] = y_chap[ind]
        
    ind_series_anom = np.unique(df_supervise[df_supervise["pred"]]["serie"].values)
    print("Séries contenant une anomalie : ", ind_series_anom)
    # matrice de confusion
    table = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize=False) # margins=True, margins_name="Total")
    table_norm = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="all")
    table_norm_index = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="index") # margins=True, margins_name="Total")
    table_norm_col = pd.crosstab(y_chap, Y_test, rownames=["Prédiction"], colnames=["Vraie valeur"], normalize="columns")
    sns.set(font_scale=1.4)  #for label size
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))
    sns.heatmap(table, annot=True, cmap="Blues", ax=ax[0, 0], annot_kws={"size": 16})  # font size
    ax[0, 0].set_title("Matrice de confusion brute")
    sns.heatmap(table_norm, annot=True, cmap="Blues", ax=ax[0, 1], annot_kws={"size": 16}) # font size
    ax[0, 1].set_title("Matrice de confusion normalisée")
    sns.heatmap(table_norm_index, annot=True, cmap="Blues", ax=ax[1, 0], annot_kws={"size": 16}) # font size
    ax[1, 0].set_title("Matrice de confusion normalisée selon les lignes")
    sns.heatmap(table_norm_col, annot=True, cmap="Blues", ax=ax[1, 1], annot_kws={"size": 16}) # font size
    ax[1, 1].set_title("Matrice de confusion normalisée selon les colonnes")
    plt.show()
    
    for num in ind_series_anom:

        fig, ax = plt.subplots(figsize=(8, 5))
        TS = df_supervise[(df_supervise["serie"] == num)]["valeurs"].values
        anoms = df_supervise[df_supervise["serie"] == num]["pred"].values
        title_obj = plt.title("Série numéro " + str(num % offset), size=25)
        for (i, x), anom in zip(enumerate(TS), anoms):

            if i > 0:  # Raccordement
                ax.plot([N * i - 1, N * i], [last, x[0]], "br"[int(anom)])

            ax.plot(range(N * i, N * (i + 1)),  x, "br"[int(anom)])
            last = x[-1]
            if anom:
                plt.setp(title_obj, color='r', fontweight="bold")
    
        plt.show()